# Poisson 1D

This notebook solves the linear problem:

$$ -u(x)'' = f(x) \text{ in } \Omega = [-1,1]$$ 

with

$$ f(x) = \frac{\pi^2}{4} \cos{\left(\frac{\pi x}{2}\right)} $$

and homogeneous Dirichlet boundary conditions using Hermite elements. The purpose is to find what the issues are while setting Dirichlet BCs and fix them. See [Manually rebuilding the mass matrix](#Manually-rebuilding-the-mass-matrix).

In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
import FIAT
%matplotlib inline
u_exact = lambda x: np.cos(np.pi*x/2)

In [ ]:
def poisson1d(V):
    deg = V.ufl_element().degree()
    def boundary(x):
        return x[0] < -1 + DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

    a = u.dx(0)*v.dx(0)*dx
    L = f*v*dx

    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
#mesh = UnitIntervalMesh(2)
#MeshTransformation.translate(mesh, Point(1.0))
mesh = IntervalMesh(2, -1, 1)

V = FunctionSpace(mesh, "Lagrange", 3)
u = poisson1d(V)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_{l2}$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_{ex}$')
pl.ylim((0,1.1))
_ = pl.legend()

# Using Hermite elements

**Fail:** first derivatives are set to 0 when DirichletBC clears all the rows of dofs of nodes at the boundary, since this includes Hermite dofs.

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d(V)

xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()

# Manually rebuilding the mass matrix

Hermite dofs are set to zero by bc.apply(): we manually find those and reset the relevant rows of the mass matrix.

In the process of writing the following two functions (`find_hermite_boundary_dofs()` and `apply_dirichlet_partially()`) it was useful to look into the code for `dolfin/fem/DirichletBC.cpp, compute_bc_pointwise()`. Also there is a [bug](https://bugs.launchpad.net/dolfin/+bug/1063868) in the automatically generated documentation which affects [`GenericMatrix.set()`](https://fenicsproject.org/olddocs/dolfin/2016.2.0/python/programmers-reference/cpp/la/GenericMatrix.html?highlight=genericmatrix#dolfin.cpp.la.GenericMatrix.set) below. Only one function with three parameters is actually exposed by SWIG and the second and last are actually the indices of the rows and columns to modify.

Note that we set the linear algebra backend to Eigen because the default PETSc uses sparse matrices whose sparsity patterns are initialised / fixed by assemble(), so we cannot freely change entries unless we directly talk to PETSc.

This can be done with petsc4py (which is available in the docker container), see [this demo](https://bitbucket.org/cwilson/dolfin/src/883fcc4a12d53c7dae1bb9d5fc7e7302051b0bae/demo/undocumented/petsc4py/python/demo_petsc4py.py?at=master&fileviewer=file-view-default), but I'm too lazy now.

**Note:** Another approach could be to "fix" DirichletBC: It seems like I need to define a special dofmap for Hermite elements which in dofmap.cell_dofs() returns only the Lagrange dofs (?). Won't this break lots of other things? Instead I could try to construct a copy of the function space with the right dofmap for the BCs and pass it to DirichletBC's constructor.

In [ ]:
def find_hermite_boundary_dofs(bc):
    """ Returns the Hermite dofs at the boundary as defined in the argument.
        bc: Boundary condition.
    """
    V = bc.function_space()
    tdim = V.element().topological_dimension()
    e = FIAT.CubicHermite(FIAT.reference_element.default_simplex(tdim), 3)
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    mask = np.array(map(lambda f: isinstance(f, FIAT.functional.PointDerivative),
                        e.dual_basis()))

    # Compute the set of hermite dofs
    hermite_dofs = set()
    for i in range(V.mesh().num_cells()):
        hermite_dofs = hermite_dofs.union(set(V.dofmap().cell_dofs(i)[mask]))
        
    # This is an easy way of retrieving the ids of dofs at the boundary:
    boundary_dofs = bc.get_boundary_values().keys()
    
    return filter(lambda x: x in boundary_dofs, hermite_dofs)


def apply_dirichlet_partially(A, b, bc):
    """ Applies DirichletBC but fixes the rows corresponding to Hermite dofs.
    Arguments:
        A: assembled mass matrix, before applying bc
        b: assembled right hand side, before applying bc
        bc: DirichletBC
    Returns:
        Nothing. Arguments are modified in place.
    """
    global parameters
    assert parameters['linear_algebra_backend'] == 'Eigen', "Need dense matrices for now."
    assert isinstance(bc, DirichletBC), "We only know how to manage Dirichlet BCs"
    
    hermite_dofs = find_hermite_boundary_dofs(bc)
    
    A0 = A.copy()
    b0 = b.copy()
    bc.apply(A, b) # Applying BCs messes things up for Hermite dofs
    
    b[hermite_dofs] = b0[hermite_dofs]
    block = A0.array()[hermite_dofs]

    rows = np.array(hermite_dofs, dtype=np.intc)
    cols = np.arange(A.size(1), dtype=np.intc) # FIXME A.size(1) is columns?

    A.set(block, rows, cols)
    A.apply("insert")


def plot_hermite_dofs(bc):
    """ Plots the mesh and marks the Hermite dofs fulfilling the boundary condition.
    Arguments:
        bc: DirichletBC.
    """
    V = bc.function_space()
    tdim = V.mesh().topology().dim()
    dofs = zip(V.dofmap().dofs(), V.tabulate_dof_coordinates().reshape((-1, tdim)))

    hdofs = find_hermite_boundary_dofs(bc)
    hdofs_coordinates = np.array([dof[1] for dof in filter(lambda p: p[0] in hdofs, dofs)])
    plot(V.mesh())
    if tdim > 1:
        pl.scatter(hdofs_coordinates[:,0], hdofs_coordinates[:,1],
                   s=15, c='red', linewidths=0, zorder=10)
    else:
        pl.ylim((-0.1,0.1))
        pl.scatter(hdofs_coordinates[:,0], np.zeros_like(hdofs_coordinates[:,0]),
                   c='red', linewidths=0, zorder=10)

In [ ]:
def poisson1d_hermite(V):
    """
        V: FunctionSpace.
    """
    assert V.ufl_element().family() == 'Hermite', 'Please pay attention.'

    def boundary(x):
        return near(x[0], -1.) or near(x[0], 1.)

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)
    plot_hermite_dofs(bc)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4",
                   degree=V.ufl_element().degree())

    a = u.dx(0)*v.dx(0)*dx
    F = f*v*dx

    A = assemble(a)
    b = assemble(F)
    apply_dirichlet_partially(A, b, bc)
    
    u = Function(V)
    U = u.vector()
    solve(A, U, b)
    return u

In [ ]:
parameters['linear_algebra_backend'] = 'Eigen'
mesh = IntervalMesh(100, -1, 1)
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d_hermite(V)

In [ ]:
xx = mesh.coordinates()
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()

# Poisson2D

We can try and use the method above in higher dimensions. However, further issues seem to enter into play. In particular, it might be that some *inner* Hermite dofs are being set to 0 by mistake. See [Interior effects](#Interior-effects).

In [ ]:
from utils import fnand, fnor, one_arg

mesh = UnitSquareMesh(10, 10)
V = FunctionSpace(mesh, "Hermite", 3)

top = lambda x: near(x[1], 1.)
right = lambda x: near(x[0], 1.)
bottom = lambda x: near(x[1], 0.)
left = lambda x: near(x[0], 0.)

# Need to project to work around lack of evaluate_dofs() for Hermite
u0 = project(Constant(0.0), V)
bc = DirichletBC(V, u0, one_arg(fnor(top, right, bottom, left)))

class td_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[1], 0.) or near(x[1], 1.)

class lr_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[0], 0.) or near(x[0], 1.)

# We will use this to mark boundaries for natural boundary conditions
exterior_facet_domains = FacetFunction("uint", mesh)
exterior_facet_domains.set_all(1)

natural_boundary = td_subdomain()
natural_boundary.mark(exterior_facet_domains, 0)
ds = ds(subdomain_data = exterior_facet_domains)

u = TrialFunction(V)
v = TestFunction(V)
f = project(Constant(4), V)
#g = project(Constant(0.), V)
a = inner(grad(u), grad(v))*dx
F = f*v*dx # + g*v*ds

# Compute solution
A = assemble(a)
b = assemble(F)
apply_dirichlet_partially(A, b, bc)
her = Function(V)
H = her.vector()
solve(A, H, b)

In [ ]:
plot_hermite_dofs(bc)

In [ ]:
pl.xlim((-0.1,1.1))
pl.ylim((-0.1,1.1))
plot(her, cmap='bone')
_ = pl.title("P3 Hermite")

## Interior effects

Notice the squiggly cross-sections in the following plot: it might be that (some of?) the Hermite dofs *inside* the domain are being set to zero...

In [ ]:
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [her([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)

## Solution with P2 Lagrange elements

In [ ]:
W = FunctionSpace(mesh, "Lagrange", 2)
u0 = Constant(0.0)
bc = DirichletBC(W, u0, one_arg(fnor(top, right, bottom, left)))

# Define variational problem
u = TrialFunction(W)
v = TestFunction(W)
f = Constant(4.)
#g = Constant(0.)
a = inner(grad(u), grad(v))*dx
F = f*v*dx #+ g*v*ds

# Compute solution
lag = Function(W)
solve(a == F, lag, bc)

In [ ]:
pl.xlim((-0.1, 1.1))
pl.ylim((-0.1, 1.1))
plot(lag, cmap='bone')
_ = pl.title("P2 Lagrange")

In [ ]:
diff = project(lag - her, V)
#diff.vector().abs()
i = 2
out =  '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
       ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
       % (i, i, norm(diff, 'L2') / norm(lag, 'L2'), i, i, norm(diff, 'H1') / norm(lag, 'H1'))
from IPython.display import Math
Math(out)

In [ ]:
_ = plot(diff, title="$u_{l%d} - u_{h3}$" % i, cmap='bone')

In [ ]:
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [lag([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)